In [1]:
import pandas as pd

# create dummies for Gender and countries 

In [129]:
df15 = pd.read_csv('data/marathon_results_2015.csv')
df16 = pd.read_csv('data/marathon_results_2016.csv')
df17 = pd.read_csv('data/marathon_results_2017.csv')
df18 = pd.read_csv('data/marathon_results_2018.csv')
df18.rename(columns={'Blank': 'Unnamed'}, inplace = True)
print(df18.shape)
df18.head()

(24087, 26)


,Unnamed: 0,Unnamed: 0.1,Bib,Name,Age,M/F,City,State,Country,Citizen,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [130]:
import numpy as np

def seconds_converter(time):
    '''Takes a time in the string form '00:00:00' and converts it into integer 
    of seconds.  Handles special '-' cases and converts them to NaNs '''
    
    if time == '-':
        return np.nan
    
    time_split = [int(x) for x in(time.split(':'))]
    return time_split[0]*(60**2) + time_split[1]*60 + time_split[2] 

def best_fit_slope(x,y):
    '''Calculate the best fit slope for all 5K splits.'''
    m = (((x.mean()*y.mean()) - (x*y).mean()) / 
         (x.mean()**2 - (x**2).mean()))
    return round(m, 2)


def race_slope(row):
    '''Create a new column of the race progression slope.'''
    nK_splits = ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']
    x = np.array([5, 10, 15, 20, 25, 30, 35, 40])
    y = np.array([row['5K']] + list(np.diff(row[nK_splits])))
    
    return best_fit_slope(x,y)

def clean_data(df):
    '''Takes in a DataFrame, drops unnecessary columns, and applies seconds_converter. '''
    
    # drop unnecessary columns
    df.drop(list(df.filter(regex='Unnamed')), axis=1, inplace=True)
    df.drop(['Citizen', 'Proj. Time', 'City'], axis=1, inplace=True)
    
    # rename columns for clarity
    df18.rename(columns={'Overall': 'overall_rank', 'Gender': 'gender_rank', 
                         'Division': 'division_rank', 'M/F': 'Gender'}, inplace = True)
    
    # convert the time to seconds on these columns
    time_cols = ['5K', '10K', '15K', '20K', 'Half', '25K', 
             '30K', '35K', '40K', 'Pace', 'Official Time']
    for col in time_cols:
        df18[col] = df18[col].apply(seconds_converter)
        
    # fill the State column with NaNs to 'none' instead so it can be 
    # concatenated with the country column
    df18['State'].fillna('', inplace=True)
    df18['state_country'] = df18['State'] + '_' + df18['Country']
    
    # create a new column progression_slope
    df18['progression_slope'] = df18.apply(race_slope, axis=1)
    
    
    
    
    df.drop(['State', 'Country', '5K', '10K', '15K', '20K', '25K', 
             '30K', '35K', '40K'], axis=1, inplace=True)
 
    return df




In [131]:
df18 = clean_data(df18)
df18.head()

,Bib,Name,Age,Gender,Half,Pace,Official Time,overall_rank,gender_rank,division_rank,state_country,progression_slope
0,189,"Ash, Alan",26,M,4606.0,358,9364,63,63,59,AL_USA,1.84
1,855,"Goodman, John Mark",38,M,4788.0,371,9712,187,186,173,AL_USA,1.51
2,1049,"Mcclung, Joshua",34,M,4805.0,375,9822,247,246,226,AL_USA,1.98
3,303,"Graham, James",26,M,4713.0,383,10027,385,377,346,AL_USA,6.95
4,5267,"Pierce, Nathaniel",31,M,5079.0,390,10219,608,593,536,AL_USA,0.48


In [127]:
test_row = df18.iloc[0]
test_row['5K']

1078.0

In [114]:

x = np.array([5, 10, 15, 20, 25, 30, 35, 40])
y = np.array([test[0]]+ list(np.diff(test)))

def best_fit_slope(x,y):
    m = (((x.mean()*y.mean()) - (x*y).mean()) / 
         (x.mean()**2 - (x**2).mean()))
    return round(m, 2)

best_fit_slope(x,y)

1.84

In [128]:
def race_slope(row):
    nK_splits = ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']
    x = np.array([5, 10, 15, 20, 25, 30, 35, 40])
    y = np.array([row['5K']] + list(np.diff(row[nK_splits])))
    
    return best_fit_slope(x,y)

df18['progression_slope'] = df18.apply(race_slope, axis=1)
df18.head()

,Bib,Name,Age,5K,10K,15K,20K,Half,25K,30K,35K,40K,Pace,Official Time,overall_rank,gender_rank,division_rank,state_country,Gender_M,progression_slope
0,189,"Ash, Alan",26,1078.0,2165.0,3258.0,4364.0,4606.0,5476.0,6619.0,7762.0,8888.0,358,9364,63,63,59,AL_USA,1,1.84
1,855,"Goodman, John Mark",38,1123.0,2261.0,3401.0,4540.0,4788.0,5673.0,6829.0,8007.0,9186.0,371,9712,187,186,173,AL_USA,1,1.51
2,1049,"Mcclung, Joshua",34,1125.0,2249.0,3399.0,4550.0,4805.0,5731.0,6914.0,8114.0,9285.0,375,9822,247,246,226,AL_USA,1,1.98
3,303,"Graham, James",26,1138.0,2244.0,3350.0,4472.0,4713.0,5589.0,6749.0,8029.0,9437.0,383,10027,385,377,346,AL_USA,1,6.95
4,5267,"Pierce, Nathaniel",31,1210.0,2409.0,3615.0,4819.0,5079.0,6023.0,7242.0,8482.0,9686.0,390,10219,608,593,536,AL_USA,1,0.48


In [118]:
df18.iloc[0][['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']]

5K     1078
10K    2165
15K    3258
20K    4364
25K    5476
30K    6619
35K    7762
40K    8888
Name: 0, dtype: object